<a href="https://colab.research.google.com/github/ekourkchi/Weather_research/blob/master/GPR_PSO_ALL_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install george

In [2]:
!pip install pyswarms


# Importing packages


In [3]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, Column 
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy import odr
import pylab as py
from matplotlib import gridspec
import sklearn.datasets as ds
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import scipy.optimize as op
from scipy.linalg import cholesky, inv,det
from scipy.optimize import minimize
import george
from george import kernels
import pandas as pd
from datetime import datetime
import time
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

In [4]:
# Import modules
import numpy as np
from pyswarms.single.global_best import GlobalBestPSO
# Import PySwarms
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [5]:
from sklearn.model_selection import train_test_split

def GPR(X, y, lnlikelihood=True):
  '''
  The output of this function is another function, either the lnlikelihood, or 
  the gp (the gaussian process regressor that is dfined by giving theta)
  '''
  n = X.shape[1]
    
  def step(theta):

        L = np.exp(theta[:n])
        sigma = np.exp(theta[n])   
        yerr = np.exp(theta[n+1])
        
        kernel = sigma * kernels.ExpSquaredKernel(np.ones(n), ndim=n)

        gp = george.GP(kernel)

        if lnlikelihood:
          gp = george.GP(kernel)
          gp.compute(X / np.vstack([L]*X.shape[0]), yerr)
       
          return -gp.lnlikelihood(y)
        else:
          X0 = X / np.vstack([L]*X.shape[0])
          gp.compute(X0, yerr)
          return gp
      
  return step

In [6]:
def metrics(y1, y2, file=None):
  '''
  y1 and y2 are two series of the same size

  This function outputs the MAE, RMSE and R^2 
  of the cross evaluated series.

  '''
  y1 = y1.reshape(-1)
  y2 = y2.reshape(-1)
  RMSE = np.sqrt(np.mean((y1-y2)**2))
  MAE = np.mean(np.abs(y1-y2))
  R2 = r2_score(y1, y2)

  if file is None:
    print('MAE: %.2f'%MAE, ' RMSE: %.2f'%RMSE, ' R^2: %.2f'%R2)
  else:
    file.write('MAE: %.2f'%MAE + ' RMSE: %.2f'%RMSE + ' R^2: %.2f'%R2 +'\n')
########################################

def funcMAX(func, X, y, addParam = 0, maxiter=500, method='L-BFGS-B', verbose=False):
  
  '''
  A function to find the optimum parameters of the input funtion "func",
  where yp = func(X) and RMSE(y-yp) is minimzed

  output: "results" is the object that contains everything about the fit
  result.x holds the optimized parameters
  '''

  t1 =  datetime.now()  # t1 and t2 are used for timing this process
  ###########################################
  n = X.shape[1]
  # Maximum Likelihood
  Param_init = np.random.rand(n+addParam)
  result = minimize(func(X, y), Param_init, 
                method=method, options={"maxiter":maxiter})
  print("--------------------")
  if verbose:
    print(result)
  ###########################################
  if not verbose: 
    print("Fit Success: ", result.success)
  t2 =  datetime.now()
  print("Execution time: ", t2-t1)
  print("--------------------")

  return result

In [7]:
def dataPrepare2(y, z, v, w, n = 3, d = 1):
  '''
  Generating discrete data points out of the given series

  y: main signal, e.g. ET0
  z, v, w: auxiliary signals (set these to zero (e.g z=y*0) if not interested)
  n: the number of previous data points that are used for forcasting
  d: the number of days ahead for forecasting

  output: feature matrix XS, and target values ys
  '''

  N = len(y)
  dd = d - 1

  XS = np.zeros((N-n-dd, n))
  ys = np.zeros(N-n-dd)

  for i in range(0, N-n-dd):
    XS[i,:n] = y[i:i+n]  
    # XS[i,n:2*n] = z[i:i+n]
    # XS[i,2*n:3*n] = v[i:i+n]   
#     XS[i,3*n:4*n] = w[i:i+n] 
    
    ys[i] = y[i+n+dd]
  # 
  return XS, ys

# Load data
data preparation
We generate the first 3 main principal components that capture the most useful information of the data. P1, P2 and P3 are not correalted with each other while they are epressed as the linear cominination of the available featurdes, i.e. ET0, VPD, Rn and T (air temperature)

**Note:** Make sure that the data file is addressed correctly and it's already avaialble in your Google drive.

In [8]:
def prepData_IR(station, mDelay, nAhead):

  data = pd.read_excel('/content/drive/MyDrive/NWdata.xlsx')

  data['TIMESTAMP'] = pd.date_range('20000101', periods=len(data), freq='D')
  data.set_index("TIMESTAMP", inplace=True)

  # resample data daily, forward linear interpolation to fill the missing values
  data.resample('1d').mean()
  data = data.interpolate(method='linear', limit_direction='forward', axis=0)

  subData = data.loc["2000":"2006-12-31"]
  N = len(subData)
  x = subData.index
  y = subData[station].ffill()
  z = y*0.
  v = y*0.
  w = y*0.


  # testing: 2013 onward
  subData_test = data.loc["2007":]
  N_test = len(subData_test)
  x_test = subData_test.index
  y_test = subData_test[station].ffill()
  z_test = y_test*0.
  v_test = y_test*0.
  w_test = y_test*0.


  XS2, ys2 = dataPrepare2(y, z, v, w, n=mDelay, d=nAhead)
  XS_test2, ys_test2 = dataPrepare2(y_test, z_test, v_test, w_test, n=mDelay, d=nAhead)

  return XS2, ys2, XS_test2, ys_test2

In [9]:
def prepData(station, mDelay, nAhead):

        dataFile = station + '.xlsx'  
        data = pd.read_excel('/content/drive/My Drive/'+ dataFile)
        ######################################################################

        # revising the column names
        for col in data.columns:
          newcol = col.split("(")[0].strip()
          data.rename(columns={col:newcol}, inplace=True)

        # setting up the index of the data frame
        data.set_index("TIMESTAMP", inplace=True)

        # resample data daily, forward linear interpolation to fill the missing values
        data.resample('1d').mean()
        data = data.interpolate(method='linear', limit_direction='forward', axis=0)

        ## Optional:
        ## Generating the first three principal (P1, P2, P3) components basesd on ET0, VPD, Rn and T.
        myData = data[["ET0", "VPD", "Rn", "T"]].ffill()
        z_scaler = StandardScaler()
        z_data = z_scaler.fit_transform(myData)
        pca_data = PCA().fit_transform(z_data);
        pca_trafo = PCA().fit(z_data);
        data["P1"] = pca_data[:,0]
        data["P2"] = pca_data[:,1]
        data["P3"] = pca_data[:,3]
        ######################################################################

        subData = data.loc["2010":"2016-12-31"]
        N = len(subData)
        x = subData.index
        y = subData["ET0"].ffill()
        z = subData["P1"].ffill()
        v = subData["P2"].ffill()
        w = subData["T"].ffill()


        # testing: 2013 onward
        subData_test = data.loc["2017":]
        N_test = len(subData_test)
        x_test = subData_test.index
        y_test = subData_test["ET0"].ffill()
        z_test = subData_test["P1"].ffill()
        v_test = subData_test["P2"].ffill()
        w_test = subData_test["T"].ffill()


        # mDelay = 5
        # nAhead = 7

        XS2, ys2 = dataPrepare2(y, z, v, w, n=mDelay, d=nAhead)
        XS_test2, ys_test2 = dataPrepare2(y_test, z_test, v_test, w_test, n=mDelay, d=nAhead)

        return XS2, ys2, XS_test2, ys_test2


# ML class


In [10]:
class psoGPR:

  def __init__(self, station, mDelay=1, nAhead=1, IR=False):
    
    self.station = station
    self.mDelay = mDelay
    self.nAhead = nAhead

    if not IR:
      self.XS2, self.ys2, self.XS_test2, self.ys_test2 = prepData(station, mDelay, nAhead)
    else:
      self.XS2, self.ys2, self.XS_test2, self.ys_test2 = prepData_IR(station, mDelay, nAhead)

    self.theta = None
    self.kf = None


  def Xi2_swarm(self, x):
    
    nParticle = x.shape[0]
    out = np.zeros(nParticle)

    for train_index, cross_index in self.kf.split(self.XS2):
      
        X_train, X_cross = self.XS2[train_index], self.XS2[cross_index]
        y_train, y_cross = self.ys2[train_index], self.ys2[cross_index]

        n = X_cross.shape[1]
        m = X_cross.shape[0]

        for n_iter in range(nParticle):
            
            theta = x[n_iter,:]

            L = np.exp(theta[:n])

            gp = GPR(X_train, y_train, lnlikelihood=False)(theta)
            gp_yp_cross, gp_yp_cross_std = gp.predict(y_train, X_cross/np.vstack([L]*m), return_var=True)

            out[n_iter] += np.sum((y_cross - gp_yp_cross)**2)

    return out


  def runSwarm(self, kFold=5, n_particles=10, n_iter=10, options={'c1': 0.5, 'c2': 0.3, 'w':0.9}):

    self.kf = KFold(n_splits=kFold)
    self.kf.get_n_splits(self.XS2)

    n_components = self.mDelay

    # Call instance of PSO
    optimizer = GlobalBestPSO(n_particles=n_particles, dimensions=n_components+2, options=options)

    # Perform optimization
    cost, theta = optimizer.optimize(self.Xi2_swarm, iters=n_iter, verbose=False)

    self.theta = theta

    return cost, theta

    
  def predict(self):

    if self.theta is None:
      return None

    truths = self.theta
    
    gp = GPR(self.XS2, self.ys2, lnlikelihood=False)(truths)

    n = self.XS2.shape[1]
    m = self.XS2.shape[0]

    m_test = self.XS_test2.shape[0]

    L = np.exp(truths[:n])

    gp_yp, gp_yp_std = gp.predict(self.ys2, self.XS2/np.vstack([L]*m), return_var=True)
    gp_yp_test, gp_yp_test_std = gp.predict(self.ys2, self.XS_test2/np.vstack([L]*m_test), return_var=True)

    return gp_yp, gp_yp_std, gp_yp_test, gp_yp_test_std


In [11]:
def runMODEL(station, mDelay = 5, nAhead = 1, kFold = 5, n_particles=2, n_iter=2, IR=False):

  # station = 'Changde'
  # mDelay = 1
  # nAhead = 1
  # kFold = 5
  # n_particles=2
  # n_iter=2
  options={'c1': 0.5, 'c2': 0.3, 'w':0.9}

  outFile = '/content/drive/My Drive/Weather_res/'+ station + '_mD%d'%mDelay + '_nA%d'%nAhead

  model = psoGPR(station, mDelay=mDelay, nAhead=nAhead, IR=IR)
  model.runSwarm(kFold=kFold, n_particles=n_particles, n_iter=n_iter, options=options)
  gp_yp, gp_yp_std, gp_yp_test, gp_yp_test_std = model.predict()

  train = {'train':model.ys2, 'predict':gp_yp, 'stdev':gp_yp_std}
  pd.DataFrame.from_dict(train).to_csv(outFile+'_train.csv')

  test = {'test':model.ys_test2, 'predict':gp_yp_test, 'stdev':gp_yp_test_std}
  pd.DataFrame.from_dict(test).to_csv(outFile+'_test.csv')

  ##############################################################################
  with open(outFile+'_metrics.txt', 'w') as file:

    file.write("station = " + station + '\n') 
    file.write("mDelay = %d\n"% mDelay) 
    file.write("nAhead = %d\n"% nAhead) 
    file.write("kFold = %d\n"% kFold) 
    file.write("n_particles = %d\n"% n_particles) 
    file.write("n_iter = %d\n"% n_iter) 

    file.write("\n")

    file.write("Training set: "+'\n')
    metrics(model.ys2, gp_yp, file=file)
    file.write("----------------------"+'\n')
    file.write("Test set: "+'\n')
    metrics(model.ys_test2, gp_yp_test, file=file)
  
  print("Done. Station: ", station)


In [12]:
STATIONS = ["Zanjan", "Tabriz", "Urmia"]

for station in STATIONS:
  runMODEL(station, mDelay = 5, nAhead = 7, kFold = 10, n_particles=20, n_iter=50, IR=True)

2020-11-22 08:27:04,820 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.


Done. Station:  Zanjan
Done. Station:  Tabriz
Done. Station:  Urmia


In [13]:
STATIONS = ["Changde", "Hailisu", "Lancang", "Miyun", "Nanmulin", "Qiemo"]

for station in STATIONS:
  runMODEL(station, mDelay = 5, nAhead = 7, kFold = 10, n_particles=20, n_iter=50)


Done. Station:  Changde
Done. Station:  Hailisu
Done. Station:  Lancang
Done. Station:  Miyun
Done. Station:  Nanmulin
Done. Station:  Qiemo
